In [1]:
import cv2
import os
import csv
import numpy as np
import imutils
import easyocr
import re

reader = easyocr.Reader(['en'], gpu=False)  # Set gpu=True if CUDA is available
image_folder = 'images'
output_csv = 'label12.csv'

def extract_plate_text(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Enhance contrast
    gray = cv2.equalizeHist(gray)

    # Reduce noise and preserve edges
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)

    # Sharpening for edge clarity
    kernel_sharp = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharp = cv2.filter2D(bfilter, -1, kernel_sharp)

    edged = cv2.Canny(sharp, 30, 200)

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:50]

    plate_img = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.018 * cv2.arcLength(contour, True), True)
        if 4 <= len(approx) <= 6:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            area = cv2.contourArea(contour)

            if 2 < aspect_ratio < 6 and area > 2000:
                plate_img = gray[y:y+h, x:x+w]
                break

    # Fallback to full image OCR if no plate detected
    ocr_target = plate_img if plate_img is not None else gray
    ocr_results = reader.readtext(ocr_target)

    # Filter and select best text
    filtered_texts = [res for res in ocr_results if re.match(r'^[A-Z0-9-]+$', res[1].replace(" ", ""))]
    
    if filtered_texts:
        best_result = max(filtered_texts, key=lambda x: x[-1])  # highest confidence
        return best_result[1]
    elif ocr_results:
        best_result = max(ocr_results, key=lambda x: x[-1])
        return best_result[1]
    else:
        return None

# Write results to CSV
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'license_plate'])

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)

            try:
                plate_text = extract_plate_text(img)
                if plate_text:
                    writer.writerow([filename, plate_text])
                    print(f"{filename}: {plate_text}")
                else:
                    writer.writerow([filename, "No text detected"])
                    print(f"{filename}: No text detected")

            except Exception as e:
                print(f"Error processing {filename}: {e}")
                writer.writerow([filename, "Error"])
 

Using CPU. Note: This module is much faster with a GPU.


0073797c-a755-4972-b76b-8ef2b31d44ab___new_IMG_20160315_071740.jpg.jpeg: KA 99hR 0212010-2011 |
00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg: 0492
018b52e6-e9a1-42c2-8ce7-0617e8c8e021___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_sbtb02_auto1.JPG: 0z7Ic 0570
03273806-bb1e-48da-8c8b-a0133a90197a___2014-Skoda-Yeti-Test-Drive.jpg.jpeg: No text detected
0369b20e-b432-4409-90f9-2420877aa386___8151536c79159a1557421da5f27f9f0e.jpg.jpeg: KLO5AK3300
07064c2c-2aa3-4419-91a4-92916de8e54c___mahindra-scorpio-old-car-500x500.jpg.jpeg: Cp
074d85b8-42ec-4d17-9b8e-9e51ae060243___hqdefault.jpg.jpeg: J 0158
07aaab79-71ee-4ea3-a9e6-640191183947___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_1208484d1392541449-nissan-terrano-official-review-img_20140215_181708.jpg: 18 P
07bd977e-d578-49a2-b345-7cee5a4db6bf___new_1031520d1356604430-skoda-rapid-joins-family-edit-sold-wp_000281.jpg.jpeg: MH12JC2813
07e787a7-6cc3-482c-9ce5-f0ce115b47f1__

In [1]:
import cv2
import numpy as np
from sklearn import svm
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import pandas as pd

# Function to load and preprocess an image
def load_image(image_path):
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray, (64, 128))
    
    return resized_image

# Function to extract HOG features
def extract_hog_features(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

# Function to get label from CSV file
def get_label(filename, labels_df):
    if filename in labels_df['filename'].values:
        label = labels_df.loc[labels_df['filename'] == filename, 'license_plate'].iloc[0]
        if label in ["Text Not Detected", "Plate Not Found"]:
            return None
        return label
    else:
        return None

# Function to load dataset
def load_dataset(folder_path, labels_df):
    images = []
    labels = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = load_image(image_path)
            
            if image is None:
                continue
            
            label = get_label(filename, labels_df)
            if label is None:
                continue
            
            images.append(image)
            labels.append(label)
    
    return images, labels
#image1.jpg,car10.jpg,
# Function to predict a new license plate
def predict_license_plate(new_image_path, clf):
    image = load_image(new_image_path)
    
    if image is None:
        print("Failed to load new image for prediction.")
        return None
    
    features = extract_hog_features(image)
    prediction = clf.predict([features])
    
    return prediction[0]



# Main function
def main():
 

    labels_df = pd.read_csv('label12.csv', encoding='latin1')

    
    folder_path = "images"
    images, labels = load_dataset(folder_path, labels_df)
    
    if len(images) == 0 or len(labels) == 0:
        print("Dataset is empty!")
        return
    
    features = [extract_hog_features(img) for img in images]
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=5)
    
    clf = svm.SVC(kernel='poly', degree=2)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy: {accuracy}")
    
    new_image_path = 'image1.jpg'
    predicted_license_plate = predict_license_plate(new_image_path, clf)
    if predicted_license_plate is not None:
        print(f"Predicted License Plate: {predicted_license_plate}")

if __name__ == "__main__":
    main()



   

Model accuracy: 0.631578947368421
Predicted License Plate: HR 26 BR 9044
